In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import joblib
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [2]:
class LOB:
    def __init__(self, wiersz):
        self.zmiana = 0
        self.dzien = wiersz[0]
        self.czas = wiersz[1]
        i = 3
        while (wiersz[i] != 'ASK'):
            i += 1
        bid = wiersz[3:i]
        ask = wiersz[(i+1):]
        self.bid = np.zeros((int((len(bid) / 2)), 2))
        self.ask = np.zeros((int((len(ask) / 2)), 2))
        for i in range(0, len(bid), 2):
            self.bid[int(i / 2)][0] = float(bid[i]) 
            self.bid[int(i / 2)][1] = float(bid[i+1])
        for i in range(0, len(ask), 2):
            self.ask[int(i / 2)] = [float(ask[i]), float(ask[i+1])]
        self.midPrice = (self.bid[len(self.bid) - 1][0] + self.ask[0][0]) / 2
    def __str__(self):
        return "Dzień: " + self.dzien + ", Czas: " + self.czas + ", midPrice: " + str(self.midPrice)

In [3]:
X = np.load("X.npy")
X1 = np.load("X1.npy")
X2 = np.load("X2.npy")
X3 = np.load("X3.npy")
X4 = np.load("X4.npy")
zmiana = np.load("zmiana.npy")
Imbalance1 = np.load("Imbalance1.npy")
indeksy = np.load("indeksy.npy")
tablica = np.load("tablica.npy")
gmmList = np.load("gmmList.npy")
tablicaCalekPo1 = np.load("tablCalekPo1.npy")
tablicaCalekPo1BezDzielenia = np.load("tabCalekPo1BezDz.npy")
tablicaTickowPo1 = np.load("tablicaTickowPo1.npy")
tablicaTickowRosnaca = np.load("tablicaTickowRosnaca.npy")
tablicaCalekRosnaca = np.load("tablicaCalekRosnaca.npy")
tablicaCalekRosnacaBezDzielenia = np.load("tablicaCalekRosnacaBezDzielenia.npy")

In [4]:
def normalizuj(tab):
    tabMean = np.mean(tab, axis = 0)
    tabStd = np.std(tab, axis = 0)
    tabStd[(tabStd == 0)] = 1
    return ((tab - tabMean) / tabStd)

def calkaBezDzielenia(a, b, gmm, eps):
    x = np.arange(a, b + 0.0000001, eps)
    y = np.exp(gmm.score_samples(x.reshape(-1, 1)))
    return np.sum(y[:x.shape[0] - 1] + y[1:])*eps

def calka(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    return (calkaBezDzielenia(a, b, gmm, eps) / 2)

def obieCalki(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    wynik = calkaBezDzielenia(a, b, gmm, eps)*(b - a)
    return wynik, wynik/n/2

def calkaBezDzieleniaF(a, b, f, eps):
    x = np.arange(a, b + 0.0000001, eps)
    y = f(x.reshape(-1, 1))
    return np.sum(y[:x.shape[0] - 1] + y[1:])*eps

def calkaF(a, b, gmm, eps):
    n = (int) ((b - a) / eps)
    return (calkaBezDzieleniaF(a, b, gmm, eps) / 2)
    
# ustawianie wartości zmian i tworzenie dla nich osobnego wektora
def generujZmiane(X):
    for i in range(len(X) - 1):
        if (X[i].midPrice > X[i + 1].midPrice):
            X[i].zmiana = 1
        if (X[i].midPrice < X[i + 1].midPrice):
            X[i].zmiana = -1
        if (X[i].midPrice == X[i + 1].midPrice):
            X[i].zmiana = 0
    return np.array([i.zmiana for i in X])

# interesują nas te loby, po którch nastąpiła zmiana mid price
def generujIndeksy(X):
    return np.arange(len(X))[np.array([(i.zmiana != 0) for i in X])]

# pierwszy sposób - imbalance to różnica najlepszego bid'a i ask'a przez ich sumę
def generujImbalance(X):
    i = 0
    Imbalance = np.zeros((X.shape[0], 1))
    for wiersz in X: 
        bid = wiersz.bid[-1, 1]
        ask = wiersz.ask[0, 1]
        Imbalance[i] = (bid - ask) / (bid + ask)
        i += 1
    return Imbalance

def generujWektor(k):
    dlugosc = imbalance.shape[0]
    ImbalanceWektor = np.zeros((dlugosc, k))
    for i in range(k, dlugosc):
        ImbalanceWektor[i] = imbalance[(i-k) : i].reshape(k)
    return ImbalanceWektor

def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidation(clf, data, dataClass, k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size / k)
    for i in range(k):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        X_train = data[mask1]
        y_train = dataClass[mask1]
        X_test = data[mask2]
        y_test = dataClass[mask2]  
        clf = clf.fit(X_train, y_train)
        if isinstance(clf, GridSearchCV):
            clf = clf.best_estimator_
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        err += testujDecisionFunction(X_test, y_test, najDecFun, clf)
    return float(err) / k

def generujPozycje(midPrice):
    pozMidPrice = (midPrice - minimum) * 10
    pozMidPriceInt = int(pozMidPrice)
    pozLewa = pozMidPriceInt
    pozPrawa = pozMidPriceInt + 1
    if (pozMidPrice == pozMidPriceInt):
        pozLewa -= 1
    return (pozLewa, pozPrawa)

def generujPrzedzialy(midPrice, ilosc, f):
    pozLewa, pozPrawa = generujPozycje(midPrice)
    przedzialy = []
    wspXowe = []
    for i in range(ilosc):
        fi = f(i)
        wspXowe.append((pozPrawa / 10) + minimum + (fi / 2))
        wspXowe.append((pozLewa / 10) + minimum - (fi / 2))
        fi10 = int(fi*10)
        tmpList = np.arange(fi10)
        przedzialy.append(pozPrawa + tmpList)
        pozPrawa += fi10
        pozLewa -= fi10
        przedzialy.append(pozLewa + 1 + tmpList)
    return przedzialy, wspXowe
        
def generujTabliceTickow(ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        wiersz = tablica[i]
        przedzialy, wspXowe = generujPrzedzialy(X[i].midPrice, ilosc, f)
        listaWspXowych.append(wspXowe)
        for j in range(2 * ilosc):
            przedzial = przedzialy[j]
            przedzial = przedzial[przedzial >= 0]
            przedzial = przedzial[przedzial < liczbaWspolrzednych]
            wynik[i][j] = np.sum(wiersz[przedzial])
    return wynik, listaWspXowych

def generujTabliceTickowDlaGMM(ilosc, f):
    wynik = np.zeros((len(X), 2 * ilosc))
    listaWspXowych = []
    for i in range(len(X)):
        wiersz = tablica[i]
        pozLewa = X[i].midPrice
        pozPrawa = pozLewa
        
        for j in range(ilosc):
            fj = f(j)
            wynik[i][2*j] = calka(pozPrawa, pozPrawa + fj, gmmList[i], 0.001)
            wynik[i][2*j + 1] = calka(pozLewa - fj, pozLewa, gmmList[i], 0.001)
            pozPrawa += fj
            pozLewa -= fj            
    return wynik

def wyliczCViWypisz(nazwa, tablica, mnoznik, liczbaKolumn, iloscIteracji):
    if mnoznik != 1:
        nazwa += '*' + str(mnoznik)
    wynik = crossValidation(svm, (((tablica[oIndeksy]).T[:liczbaKolumn]).T)*mnoznik, zmiana, iloscIteracji)
    print(nazwa, "zredukowana do ", liczbaKolumn, " kolumn, Cross-Vall score: ", wynik)
    
def funkcjaPrzedzialow(i):
    return (int((0.1 + (0.05 + i / 20) * np.sqrt(i)) * 10) / 10)
    
minimum = 100000
maksimum = 0
for wiersz in X:
    bid = wiersz.bid[0, 0]
    ask = wiersz.ask[-1, 0]
    if(ask > maksimum):
        maksimum = ask
    if(bid < minimum):
        minimum = bid 
liczbaWspolrzednych = int((maksimum - minimum) * 10 + 1)
    

In [5]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def crossValidation(clf, data, dataClass, k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size / k)
    for i in range(k):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        X_train = data[mask1]
        y_train = dataClass[mask1]
        X_test = data[mask2]
        y_test = dataClass[mask2]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        err += testujDecisionFunction(X_test, y_test, najDecFun, clf)
    return float(err) / k

def crossValidationPoKolei(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

In [6]:
def czas(h, m):
    mnoznikM = 100000
    mnoznikH = 10000000
    return h*mnoznikH + m*mnoznikM

def czasOdwrotny(x):
    mnoznikM = 100000
    mnoznikH = 10000000
    return (int(x / mnoznikH), int((x % mnoznikH) / mnoznikM))

for i in range (len(X)):
    X[i].czas = int(X[i].czas)
    X[i].dzien = int(X[i].dzien)
    
# wszystko było na przełomie 2013.09, więc obchodzi nas tylko dzień
for i in range (len(X)):
    X[i].dzien = X[i].dzien % 100
    
zmiana = generujZmiane(X)
maskaCzasu = np.array([wiersz.czas > czas(8, 0) and wiersz.czas < czas(16, 45) for wiersz in X])
oX = X[maskaCzasu]
indeksy = generujIndeksy(X)
oIndeksy = []
for indeks in indeksy:
    if (maskaCzasu[indeks]):
        oIndeksy.append(indeks)
oIndeksy = np.array(oIndeksy)
imbalance = generujImbalance(X)

In [6]:
param_grid = {'C': [1, 1e1, 1e2, 1e3, 1e4],
              'gamma': [0.0001, 0.001, 0.01, 0.1], 
             }
gamma = 1
kernel = 'rbf'
C = 1
class_weight = 'balanced' 
svm = SVC(kernel='rbf', class_weight='balanced', )

In [47]:
def czas(h, m):
    mnoznikM = 100000
    mnoznikH = 10000000
    return h*mnoznikH + m*mnoznikM

def czasOdwrotny(x):
    mnoznikM = 100000
    mnoznikH = 10000000
    return (int(x / mnoznikH), int((x % mnoznikH) / mnoznikM))

In [48]:
for i in range (len(X)):
    X[i].czas = int(X[i].czas)
    X[i].dzien = int(X[i].dzien)

In [49]:
# wszystko było na przełomie 2013.09, więc obchodzi nas tylko dzień
for i in range (len(X)):
    X[i].dzien = X[i].dzien % 100

In [10]:
X[0].dzien # jakis błąd, nie warto patrzeć

1

In [51]:
X[3].czas

75200000

In [71]:
for k in range(0, 17):
    maska = np.array([(wiersz.dzien == k) for wiersz in X])
    ilosc = np.sum(maska)
    print(k, ilosc)

0 0
1 1
2 545
3 547
4 545
5 550
6 549
7 0
8 0
9 548
10 549
11 546
12 548
13 546
14 0
15 0
16 0


In [11]:
listaDni = np.array([2, 3, 4, 5, 6, 9, 10, 11, 12 ,13])

In [85]:
procenty = np.zeros((14, 24))
maska
for k in listaDni:
    maska = np.array([(wiersz.dzien == k) for wiersz in X])
    ilosc = np.max([np.sum(maska), 1])
    XDlaDnia = X[maska]
    for i in range(24):
        procenty[k-1][i] = (np.sum([(wiersz.czas >= czas(i, 0) and wiersz.czas < czas(i+1,0)) for wiersz in XDlaDnia]) / ilosc * 100)

In [92]:
naglowek = ''
for i in range(6, 20):
    naglowek += '{0}  '.format(i)
print(naglowek)
print()
for k in (listaDni-1):
    wiersz = ''
    for i in range(6, 20):
        wiersz += '{0:.0f}  '.format(procenty[k][i])
    print(wiersz)

6  7  8  9  10  11  12  13  14  15  16  17  18  19  

0  2  11  11  11  11  11  11  11  11  9  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  0  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  0  0  0  
0  2  11  11  11  11  11  11  11  11  10  1  0  0  
0  2  11  11  11  11  11  11  11  11  10  0  0  0  


In [103]:
naglowek = ''
for i in range(6, 20):
    naglowek += '{0}  '.format(i)
print(naglowek)
print()

wiersz = ''
wynikProcentow = np.sum(procenty[(listaDni-1)], axis = 0) / 10
for i in range(6, 20):
    wiersz += '{0:.0f}  '.format(wynikProcentow[i])
print(wiersz)

6  7  8  9  10  11  12  13  14  15  16  17  18  19  

0  2  11  11  11  11  11  11  11  11  10  1  0  0  


In [108]:
ciekaweGodziny = [czas(7, 30), czas(8, 0), czas(8, 30), czas(16, 0), czas(16, 30), czas(17, 0)]
procentyCiekawe = np.zeros((20, len(ciekaweGodziny)))

maska
for k in listaDni:
    maska = np.array([(wiersz.dzien == k) for wiersz in X])
    ilosc = np.max([np.sum(maska), 1])
    XDlaDnia = X[maska]
    i = 0
    for godzina in ciekaweGodziny:
        procentyCiekawe[k-1][i] = (np.sum([(wiersz.czas >= godzina and wiersz.czas < (godzina + czas(0, 30))) for wiersz in XDlaDnia]) / ilosc * 100)
        i += 1

In [120]:
naglowek = ''
for i in range(6, 20):
    naglowek += '{0}  '.format(i)
print([czasOdwrotny(godzina) for godzina in ciekaweGodziny])
print()
for k in (listaDni-1):
    wiersz = ''
    for i in range(len(ciekaweGodziny)):
        wiersz += '{0:.1f}         '.format(procentyCiekawe[k][i])
    print(wiersz)

[(7, 30), (8, 0), (8, 30), (16, 0), (16, 30), (17, 0)]

1.7         5.5         5.5         5.5         3.9         0.6         
1.6         5.5         5.5         5.5         4.6         0.4         
1.7         5.5         5.5         5.5         4.0         0.6         
1.6         5.5         5.5         5.5         4.9         0.5         
1.6         5.5         5.5         5.5         4.6         0.7         
1.6         5.5         5.5         5.5         4.4         0.7         
1.6         5.5         5.5         5.5         4.4         0.9         
1.6         5.5         5.5         5.5         4.4         0.4         
1.6         5.5         5.5         5.5         4.4         0.7         
1.6         5.5         5.5         5.5         4.6         0.4         


In [121]:
ciekaweGodziny = [czas(7, 30), czas(7, 45),czas(16, 30), czas(16, 45)]
procentyCiekawe = np.zeros((20, len(ciekaweGodziny)))

maska
for k in listaDni:
    maska = np.array([(wiersz.dzien == k) for wiersz in X])
    ilosc = np.max([np.sum(maska), 1])
    XDlaDnia = X[maska]
    i = 0
    for godzina in ciekaweGodziny:
        procentyCiekawe[k-1][i] = (np.sum([(wiersz.czas >= godzina and wiersz.czas < (godzina + czas(0, 15))) for wiersz in XDlaDnia]) / ilosc * 100)
         i += 1
naglowek = ''
for i in range(6, 20):
    naglowek += '{0}  '.format(i)
print([czasOdwrotny(godzina) for godzina in ciekaweGodziny])
print()
for k in (listaDni-1):
    wiersz = ''
    for i in range(len(ciekaweGodziny)):
        wiersz += '{0:.1f}         '.format(procentyCiekawe[k][i])
    print(wiersz)

[(7, 30), (7, 45), (16, 30), (16, 45)]

0.0         1.7         2.0         1.8         
0.0         1.6         2.6         2.0         
0.0         1.7         2.6         1.5         
0.0         1.6         2.7         2.2         
0.0         1.6         2.7         1.8         
0.0         1.6         2.6         1.8         
0.0         1.6         2.7         1.6         
0.0         1.6         2.6         1.8         
0.0         1.6         2.7         1.6         
0.0         1.6         2.6         2.0         


In [122]:
# czyli sensowne godziny to 8-16:45

In [33]:
np.sum([wiersz.czas > czas(8, 0) and wiersz.czas < czas(16, 45) for wiersz in X]) / X.shape[0]

0.95560833028863723

In [63]:
def czas(h, m):
    mnoznikM = 100000
    mnoznikH = 10000000
    return h*mnoznikH + m*mnoznikM

def czasOdwrotny(x):
    mnoznikM = 100000
    mnoznikH = 10000000
    return (int(x / mnoznikH), int((x % mnoznikH) / mnoznikM))

for i in range (len(X)):
    X[i].czas = int(X[i].czas)
    X[i].dzien = int(X[i].dzien)
    
# wszystko było na przełomie 2013.09, więc obchodzi nas tylko dzień
for i in range (len(X)):
    X[i].dzien = X[i].dzien % 100

In [73]:
zmiana = generujZmiane(X)
maskaCzasu = np.array([wiersz.czas > czas(8, 0) and wiersz.czas < czas(16, 45) for wiersz in X])
oX = X[maskaCzasu]
indeksy = generujIndeksy(X)
oIndeksy = []
for indeks in indeksy:
    if (maskaCzasu[indeks]):
        oIndeksy.append(indeks)
oIndeksy = np.array(oIndeksy)
imbalance = generujImbalance(X)

TypeError: only integer arrays with one element can be converted to an index

In [69]:
indeksy.shape[0] / X.shape[0]

0.8498355864084765

In [61]:
w= 0
for i in X:
    if i.zmiana !=0:
        w +=1
w

0

In [70]:
daneS = 'OrderBookSnapshots.csv';
with open(daneS, 'r') as f:
    dane = f.readlines()
X = []    

for wiersz in dane:
    newWiersz = wiersz.split()
    X.append(LOB(newWiersz))
    

for i in range (len(X)):
    X[i].czas = int(X[i].czas)
    X[i].dzien = int(X[i].dzien)
for i in range (len(X)):
    X[i].dzien = X[i].dzien % 100
zmiana = generujZmiane(X)

In [13]:
gamma = 0.0000001
kernel = 'rbf'
C = 10000
tol = 0.001
svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
print("gamma:", gamma, ", kernel:", kernel, ", C:", C, ", tol:", tol)
wielkiTest(indeksy, "imbalance", imbalance, svm, ilosc = 1, procent = 0.4, iloscKolumn = 120)

gamma: 1e-07 , kernel: rbf , C: 10000 , tol: 0.001


NameError: name 'wielkiTest' is not defined

In [28]:
k = 4
n = int(size / (k+1))
trainCount = int(2 * n * p)
for i in range(k):
    trainMask = np.arange(i*n, i*n + trainCount)
    testMask = np.arange(i*n + trainCount, (i+2)*n)

'linear'

In [57]:
oIndeksy.shape[0], indeksy.shape[0]

(4618, 4652)

In [14]:
gamma = 0.001
kernel = 'rbf'
C = 1
tol = 0.001
# class_weight='balanced', 
svm = SVC(gamma = gamma, kernel = kernel, C = C, tol = tol)
print("gamma:", gamma, ", kernel:", kernel, ", C:", C, ", tol:", tol)
wielkiTest(oIndeksy, "imbalance", imbalance, svm, ilosc = 4, procent = 0.4, iloscKolumn = 120)

gamma: 0.001 , kernel: rbf , C: 1 , tol: 0.001


NameError: name 'wielkiTest' is not defined

In [21]:
def test(indeksy, nazwa, tablica, tablica2, tablica3, svm, ilosc = 4, procent = 0.75, iloscKolumn = 120):
    wynik1 = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik2 = crossValidationPoKolei(svm, (((tablica2[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik3 = crossValidationPoKolei(svm, (((tablica3[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 
    print("Czyste dane:", wynik1, ", S+N:", wynik2, ", N+S:", wynik3)

In [150]:
def test2(indeksy, nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, ilosc = 4, procent = 0.75, iloscKolumn = 120):
    wynik1 = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik2 = crossValidationPoKolei(svm, (((tablica2[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik3 = crossValidationPoKolei(svm, (((tablica3[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent)
    wynik4= crossValidationPoKolei(svm, (((tablica4[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 
    wynik5 = crossValidationPoKolei(svm, (((tablica5[indeksy]).T[:iloscKolumn]).T), zmiana[indeksy], ilosc, procent) 

    return wynik1, wynik2, wynik3, wynik4, wynik5

In [23]:
gamma = 0.001
kernel = 'linear'
C = 1
tol = 0.001
svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)

imbalanceSN = prep.Normalizer().fit_transform(prep.StandardScaler().fit_transform(imbalance))
imbalanceNS = prep.StandardScaler().fit_transform(prep.Normalizer().fit_transform(imbalance))

print("gamma:", gamma, ", kernel:", kernel, ", C:", C, ", tol:", tol)
test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

gamma: 0.001 , kernel: linear , C: 1 , tol: 0.001
Czyste dane: 0.5354851643945469 , S+N: 0.5425020048115476 , N+S: 0.541299117882919


In [151]:
gammas = [1e-5, 1e-2, 1, 1e2, 1e5]
kernels = ['linear', 'rbf']
Cs = [1e-5, 1e-2, 1, 1e2, 1e5]
tols = [1e-5, 1e-2, 1, 1e2, 1e5]


imbalanceS = prep.StandardScaler().fit_transform(imbalance)
imbalanceN = prep.Normalizer().fit_transform(imbalance)
imbalanceSN = prep.Normalizer().fit_transform(imbalanceS)
imbalanceNS = prep.StandardScaler().fit_transform(imbalanceN)
procent = 0.33
print("procent= ", procent)
print('Czyste  S     N     S->N    N->S')
for gamma in gammas:
    for C in Cs:
        for tol in tols:
            for kernel in kernels:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                wynik1, wynik2, wynik3, wynik4, wynik5 = test2(oIndeksy, "imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = procent, iloscKolumn = 120)
                odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                if (wynik1 > 0.53 or wynik2 > 0.53 or wynik3 > 0.53 or wynik4 > 0.53 or wynik5 > 0.53):
                    odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

procent=  0.33
Czyste  S     N     S->N    N->S
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1e-05
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1e-05
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: linear C: 1e-05 tol: 0.01
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 0.01
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1
0.5008  0.5008  0.5008  0.5008  0.5008 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100.0
0.5008  0.5008  0.5008  0.5008  0.5008 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100.0
0.5008  0.5008  0.5008  0.5008  0.5008 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100000.0
0.5008  0.5008  0.5008  0.5008  0.5008 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100000.0
"0.5346  0.5346  0.5416  0.5428  0.5416" - dla: g: 1e-05 k: linear C: 0.01 tol: 1e-05
"0.5346  0.5346  0.5416

In [167]:
svm = SVC(class_weight='balanced', gamma = 0.001, kernel = 'rbf', C = 1, tol = 0.001)
wypisz = test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.5354851643945469 , S+N: 0.5425020048115476 , N+S: 0.541299117882919


In [157]:
svm = SVC(class_weight='balanced', gamma = 0.01, kernel = kernel, C = 1, tol = tol)
print("gamma:", gamma, ", kernel:", kernel, ", C:", C, ", tol:", tol)
test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

gamma: 10000.0 , kernel: rbf , C: 10000.0 , tol: 10000.0
Czyste dane: 0.4991980753809142 , S+N: 0.4991980753809142 , N+S: 0.4991980753809142


In [168]:
svm = SVC(class_weight='balanced', gamma = 0.001, kernel = 'rbf', C = 1, tol = 0.000001)
wypisz = test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.5354851643945469 , S+N: 0.5425020048115476 , N+S: 0.541299117882919


In [169]:
svm = SVC(class_weight='balanced', gamma = 0.001, kernel = 'rbf', C = 1, tol = 0.0001)
wypisz = test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.5354851643945469 , S+N: 0.5425020048115476 , N+S: 0.541299117882919


In [47]:
svm = SVC(class_weight='balanced', gamma = 1e10, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=4000)
wypisz = test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.49899759422614276 , S+N: 0.5425020048115476 , N+S: 0.541299117882919


In [25]:
svm = SVC(class_weight='balanced', gamma =1e-3, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=4000)
wypisz = test(oIndeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.5345594179466451 , S+N: 0.5428455941794664 , N+S: 0.5416329830234438


In [48]:
svm = SVC(class_weight='balanced', gamma = 0.001, kernel = 'rbf', C = 1, tol = 0.0001)
wypisz = test(indeksy, "imbalance", imbalance, imbalanceSN, imbalanceNS, svm, ilosc = 2, procent = 0.33, iloscKolumn = 120)

Czyste dane: 0.5500722195474241 , S+N: 0.5474241694752047 , N+S: 0.5464612421762157


In [52]:
# teraz zmienię podejście - będe przekazywał listę zbiorów na których przetestować svm i p - ile procent mają stanowić dane uczące


In [16]:
def dzien(x):
    return {
        0 : 'poniedziałek',
        1 : 'wtorek',
        2 : 'środa',
        3 : 'czwartek',
        4 : 'piątek'
    }[x % 5]

In [17]:
def crossValidationPrzedzialy(clf, data, listaIndeksow, p):
    dataClass = zmiana
    err = 0
    i = 0
    listaWynikow = []
    for indeksy in listaIndeksow:
        n = indeksy.shape[0]
        trainCount = int(p*n)
        trainMask = indeksy[np.arange(trainCount)]
        testMask = indeksy[np.arange(trainCount, n)]
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        listaWynikow.append(wynik)
        err += wynik
        i += 1
    return float(err) / len(listaIndeksow), listaWynikow

def wyliczCVPoKoleiiWypisz(indeksy, nazwa, tablica, iloscIteracji, p, liczbaKolumn, svm):
    wynik = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:liczbaKolumn]).T), zmiana[indeksy], iloscIteracji, p)
    print(nazwa, "zredukowana do ", liczbaKolumn, " kolumn, Cross-Vall score: ", wynik)
    print()
    
param_grid = {'C': [1e-5, 1e-2, 1, 1e2, 1e5],
              'gamma': [1e-5, 1e-2, 1, 1e2, 1e5],
             'kernel': ['rbf', 'linear']}
svm = GridSearchCV(SVC(class_weight='balanced', cache_size = 2000), param_grid, n_jobs = 6)

imbalanceS = prep.StandardScaler().fit_transform(imbalance)
imbalanceN = prep.Normalizer().fit_transform(imbalance)
imbalanceSN = prep.Normalizer().fit_transform(imbalanceS)
imbalanceNS = prep.StandardScaler().fit_transform(imbalanceN)

def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialy(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik2, listaWynikow2 = crossValidationPrzedzialy(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik3, listaWynikow3 = crossValidationPrzedzialy(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik4, listaWynikow4 = crossValidationPrzedzialy(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    wynik5, listaWynikow5 = crossValidationPrzedzialy(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    print('Czyste    S       N      S->N    N->S')
    for i in range(10):
        print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}    - {5}, tydzień {6}".format(listaWynikow1[i], listaWynikow2[i], listaWynikow3[i], listaWynikow4[i], listaWynikow5[i], dzien(i), int(i/5) + 1))
    print("Wyniki średnie:")
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5))
    

In [18]:
indeksyDni = []
for dzien in listaDni:
    indeksyDnia = []
    for indeks in oIndeksy:
        if (X[indeks].dzien == dzien):
            indeksyDnia.append(indeks)
    indeksyDni.append(np.array(indeksyDnia))

In [130]:
for procent in (np.arange(0.1, 1, 0.05)):
    svm = SVC(class_weight='balanced', gamma = 1e-4, kernel = 'rbf', C = 1, tol = 1e-5, cache_size=4000)
    print("procent danych uczących: ", procent,  "SVM: gamma:", svm.gamma, ", kernel:", svm.kernel, ", C:", svm.C, ", tol:", svm.tol)
    test("imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, indeksyDni, procent = procent)
    print()

procent danych uczących:  0.1 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
Tydzień 1
0.5076  0.5076  0.5328  0.5303  0.5328    - poniedziałek, tydzień 1
0.4432  0.4432  0.4432  0.4409  0.4432    - wtorek, tydzień 1
0.5324  0.5324  0.5602  0.5602  0.5602    - środa, tydzień 1
0.5157  0.5157  0.5422  0.5398  0.5422    - czwartek, tydzień 1
0.5512  0.5512  0.5628  0.5628  0.5628    - piątek, tydzień 1
0.5133  0.5133  0.4891  0.4843  0.4891    - poniedziałek, tydzień 2
0.5464  0.5464  0.5489  0.5489  0.5489    - wtorek, tydzień 2
0.5264  0.5264  0.5337  0.5385  0.5337    - środa, tydzień 2
0.5061  0.5061  0.5230  0.5230  0.5230    - czwartek, tydzień 2
0.5233  0.5233  0.4447  0.4472  0.4447    - piątek, tydzień 2
Wyniki średnie:
0.5166  0.5166  0.5181  0.5176  0.5181

procent danych uczących:  0.15 SVM: gamma: 0.0001 , kernel: rbf , C: 1 , tol: 1e-05
Czyste    S       N      S->N    N->S
Tydzień 1
0.5027  0.5027  0.5374  0.5348  0.5374    - pon

In [19]:
def test(nazwa, tablica, tablica2, tablica3, tablica4, tablica5, svm, listZbiorowIndeksow, procent = 0.75, iloscKolumn = 60):
    wynik1, listaWynikow1 = crossValidationPrzedzialy(svm, (((tablica).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik2, listaWynikow2 = crossValidationPrzedzialy(svm, (((tablica2).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik3, listaWynikow3 = crossValidationPrzedzialy(svm, (((tablica3).T[:iloscKolumn]).T), listZbiorowIndeksow, procent)
    wynik4, listaWynikow4 = crossValidationPrzedzialy(svm, (((tablica4).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    wynik5, listaWynikow5 = crossValidationPrzedzialy(svm, (((tablica5).T[:iloscKolumn]).T), listZbiorowIndeksow, procent) 
    return wynik1, wynik2, wynik3, wynik4, wynik5

In [148]:
gammas = [1e-5, 1e-2, 1, 1e2, 1e5]
kernels = ['linear', 'rbf']
Cs = [1e-5, 1e-2, 1, 1e2, 1e5]
tols = [1e-5, 1e-2, 1, 1e2, 1e5]

procent = 0.7
print("procent =", procent)
print('Czyste  S     N     S->N    N->S')
for gamma in gammas:
    for C in Cs:
        for tol in tols:
            for kernel in kernels:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                wynik1, wynik2, wynik3, wynik4, wynik5 = test("imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, indeksyDni, procent = procent)
                odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                if (wynik1 > 0.53 or wynik2 > 0.53 or wynik3 > 0.53 or wynik4 > 0.53 or wynik5 > 0.53):
                    odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                
                print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

Czyste  S     N     S->N    N->S
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1e-05
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1e-05
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: linear C: 1e-05 tol: 0.01
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 0.01
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1
0.5065  0.5065  0.5065  0.5065  0.5065 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100.0
0.5065  0.5065  0.5065  0.5065  0.5065 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100.0
0.5065  0.5065  0.5065  0.5065  0.5065 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100000.0
0.5065  0.5065  0.5065  0.5065  0.5065 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100000.0
"0.5119  0.5119  0.5473  0.5382  0.5473" - dla: g: 1e-05 k: linear C: 0.01 tol: 1e-05
"0.5119  0.5119  0.5473  0.5382  0.547

In [152]:
gammas = [1e-5, 1e-2, 1, 1e2, 1e5]
kernels = ['linear', 'rbf']
Cs = [1e-5, 1e-2, 1, 1e2, 1e5]
tols = [1e-5, 1e-2, 1, 1e2, 1e5]


imbalanceS = prep.StandardScaler().fit_transform(imbalance)
imbalanceN = prep.Normalizer().fit_transform(imbalance)
imbalanceSN = prep.Normalizer().fit_transform(imbalanceS)
imbalanceNS = prep.StandardScaler().fit_transform(imbalanceN)
procent = 0.7
print("procent= ", procent)
print('Czyste  S     N     S->N    N->S')
for gamma in gammas:
    for C in Cs:
        for tol in tols:
            for kernel in kernels:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                wynik1, wynik2, wynik3, wynik4, wynik5 = test2(oIndeksy, "imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, ilosc = 4, procent = procent, iloscKolumn = 120)
                odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                if (wynik1 > 0.53 or wynik2 > 0.53 or wynik3 > 0.53 or wynik4 > 0.53 or wynik5 > 0.53):
                    odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

procent=  0.7
Czyste  S     N     S->N    N->S
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1e-05
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1e-05
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: linear C: 1e-05 tol: 0.01
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 0.01
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: linear C: 1e-05 tol: 1
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100.0
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100.0
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100000.0
0.5023  0.5023  0.5023  0.5023  0.5023 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100000.0
"0.5393  0.5393  0.5338  0.5347  0.5338" - dla: g: 1e-05 k: linear C: 0.01 tol: 1e-05
"0.5393  0.5393  0.5338 

In [153]:
gammas = [1e-5, 1e-2, 1, 1e2, 1e5]
kernels = ['linear', 'rbf']
Cs = [1e-5, 1e-2, 1, 1e2, 1e5]
tols = [1e-5, 1e-2, 1, 1e2, 1e5]

procent = 0.3
print("procent =", procent)
print('Czyste  S     N     S->N    N->S')
for gamma in gammas:
    for C in Cs:
        for tol in tols:
            for kernel in kernels:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                wynik1, wynik2, wynik3, wynik4, wynik5 = test("imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, indeksyDni, procent = procent)
                odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                if (wynik1 > 0.53 or wynik2 > 0.53 or wynik3 > 0.53 or wynik4 > 0.53 or wynik5 > 0.53):
                    odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                
                print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

procent = 0.3
Czyste  S     N     S->N    N->S
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: linear C: 1e-05 tol: 1e-05
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1e-05
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: linear C: 1e-05 tol: 0.01
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 0.01
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: linear C: 1e-05 tol: 1
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-05 k: linear C: 1e-05 tol: 100000.0
0.5032  0.5032  0.5032  0.5032  0.5032 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 100000.0
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: linear C: 0.01 tol: 1e-05
0.5246  0.5246  0.5246  0.5234  0.5246

In [ ]:
gammas = [1e-5]
kernels = ['linear', 'rbf']
Cs = [1e-5, 1]
tols = [1e-20]

procent = 0.3
print("procent =", procent)
print('Czyste  S     N     S->N    N->S')
for gamma in gammas:
    for C in Cs:
        for tol in tols:
            for kernel in kernels:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = kernel, C = C, tol = tol)
                wynik1, wynik2, wynik3, wynik4, wynik5 = test("imbalance", imbalance, imbalanceS, imbalanceN, imbalanceSN, imbalanceNS, svm, indeksyDni, procent = procent)
                odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                if (wynik1 > 0.53 or wynik2 > 0.53 or wynik3 > 0.53 or wynik4 > 0.53 or wynik5 > 0.53):
                    odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
                
                print(odpowiedz, "- dla: g:", gamma, "k:", kernel, "C:", C, "tol:", tol)  

procent = 0.3
Czyste  S     N     S->N    N->S
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: linear C: 1e-05 tol: 1e-20
0.5246  0.5246  0.5246  0.5234  0.5246 - dla: g: 1e-05 k: rbf C: 1e-05 tol: 1e-20


In [20]:
# testowania dla danego dnia jednego tygodnia uczac sie na tym samym dniu w innym tygodniu

def crossValidationPrzedzialyPary(clf, data, listaIndeksow):
    dataClass = zmiana
    err = 0
    i = 0
    listaWynikow = []
    for indeksy in listaIndeksow:
        trainMask = indeksy[0]
        testMask = indeksy[1]
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        listaWynikow.append(wynik)
        err += wynik
        i += 1
    return float(err) / len(listaIndeksow), listaWynikow

def wyliczCVPoKoleiiWypisz(indeksy, nazwa, tablica, iloscIteracji, p, liczbaKolumn, svm):
    wynik = crossValidationPoKolei(svm, (((tablica[indeksy]).T[:liczbaKolumn]).T), zmiana[indeksy], iloscIteracji, p)
    print(nazwa, "zredukowana do ", liczbaKolumn, " kolumn, Cross-Vall score: ", wynik)
    print()
    
param_grid = {'C': [1e-5, 1e-2, 1, 1e2, 1e5],
              'gamma': [1e-5, 1e-2, 1, 1e2, 1e5],
             'kernel': ['rbf', 'linear']}
svm = GridSearchCV(SVC(class_weight='balanced', cache_size = 2000), param_grid, n_jobs = 6)

imbalanceS = prep.StandardScaler().fit_transform(imbalance)
imbalanceN = prep.Normalizer().fit_transform(imbalance)
imbalanceSN = prep.Normalizer().fit_transform(imbalanceS)
imbalanceNS = prep.StandardScaler().fit_transform(imbalanceN)

-

In [21]:
indeksyDni = []
for dzien in listaDni:
    indeksyDniaUczace = []
    indeksyDniaTestowe =[]
    i = 0
    for indeks in oIndeksy:
        if (i == 5):
            break
        if (X[indeks].dzien == dzien):
            indeksyDnia.append(indeks)
        i += 1
    indeksyDni.append(np.array(indeksyDnia))

In [1]:
print(3)

3


In [7]:
def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

In [8]:
listaTablicaCalekPo1 = standaryzowanie(tablicaCalekPo1)
listaImbalance = standaryzowanie(imbalance)
listaTablicaCalekRosnaca = standaryzowanie(tablicaCalekRosnaca)
listaTablicaTickowPo1 = standaryzowanie(tablicaTickowPo1)
listaTablicaTickowRosnaca = standaryzowanie(tablicaTickowRosnaca)

In [9]:
oListaTablicaCalekPo1 = standaryzowanie(tablicaCalekPo1[oIndeksy])
oListaImbalance = standaryzowanie(imbalance[oIndeksy])
oListaTablicaCalekRosnaca = standaryzowanie(tablicaCalekRosnaca[oIndeksy])
oListaTablicaTickowPo1 = standaryzowanie(tablicaTickowPo1[oIndeksy])
oListaTablicaTickowRosnaca = standaryzowanie(tablicaTickowRosnaca[oIndeksy])

In [10]:
def crossValidationGroupingPoKolei(clf, data, dataClass, k, p, kmeans, kmeansData):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    wynik = 0
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        
        xKmeansTrain = kmeansData[trainMask]
        xKmeansTest = kmeansData[testMask]
        
        wynik += przelicz(X_train, X_test, y_train, y_test, kmeans, clf, xKmeansTrain, xKmeansTest)
        
    return np.sum(wynik) / k

def przelicz(xTrain, xTest, yTrain, yTest, kmeans, clf, xKmeansTrain, xKmeansTest):
    kmeans.fit(xKmeansTrain)
    trainLabels = kmeans.labels_
    testLabels = kmeans.predict(xKmeansTest)
    k = kmeans.n_clusters
    n = xTrain.shape[0]
    wynik = 0
    for i in range(k):
        if (np.sum(trainLabels == i) > 0 and np.sum(testLabels == i) > 0):
            wynik += licz(xTrain[trainLabels==i], xTest[testLabels==i], yTrain[trainLabels==i], yTest[testLabels==i], clf)
    return wynik / k

def licz(xTrain, xTest, yTrain, yTest, clf):
    clf = clf.fit(xTrain, yTrain)
    yTrain = yTrain.reshape((yTrain.shape[0], 1))
    wynikDecFun = clf.decision_function(xTrain).reshape(yTrain.shape)
    najDecFun = najlepszeDecFun(yTrain, wynikDecFun)  
    return testujDecisionFunction(xTest, yTest, najDecFun, clf)

def stworzGrupowanie(kmeans, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    wynik = []
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        y_train = dataClass[trainMask]
        y_test = dataClass[testMask]  
        
        xKmeansTrain = kmeansData[trainMask]
        xKmeansTest = kmeansData[testMask]
        kmeans.fit(xKmeansTrain)
        trainLabels = kmeans.labels_
        testLabels = kmeans.predict(xKmeansTest)
        wynik.append([trainLabels, testLabels])
    return wynik

    

In [11]:
def crossValidationPoKolei(clf, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        clf = clf.fit(X_train, y_train)
        y_train = y_train.reshape((y_train.shape[0], 1))
        wynikDecFun = clf.decision_function(X_train).reshape(y_train.shape)
        najDecFun = najlepszeDecFun(y_train, wynikDecFun)        
        wynik = testujDecisionFunction(X_test, y_test, najDecFun, clf)
        err += wynik
    return float(err) / k

def crossValidationGroupingPoKolei(clf, data, dataClass, k, p, kmeans, kmeansData):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    wynik = 0
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        X_train = data[trainMask]
        y_train = dataClass[trainMask]
        X_test = data[testMask]
        y_test = dataClass[testMask]  
        
        wynik += przelicz(X_train, X_test, y_train, y_test, kmeans, clf, kmeansData[i])
        
    return np.sum(wynik) / k

def przelicz(xTrain, xTest, yTrain, yTest, kmeans, clf, kmeansData):
    trainLabels = kmeansData[0]
    testLabels = kmeansData[1]
    k = kmeans.n_clusters
    wynik = 0
    for i in range(k):
        iloscTestowych = np.sum(testLabels == i)
        if (np.sum(trainLabels == i) > 1 and iloscTestowych > 1):
            wynik += iloscTestowych * licz(xTrain[trainLabels==i], xTest[testLabels==i], yTrain[trainLabels==i], yTest[testLabels==i], clf)
    return wynik / yTest.shape[0]

def licz(xTrain, xTest, yTrain, yTest, clf):
    if (np.unique(yTrain).shape[0] == 1):
        wynik = np.sum(yTest == yTrain[0]) / yTest.shape[0]
        return wynik
    
    clf = clf.fit(xTrain, yTrain)
    yTrain = yTrain.reshape((yTrain.shape[0], 1))
    wynikDecFun = clf.decision_function(xTrain).reshape(yTrain.shape)
    najDecFun = najlepszeDecFun(yTrain, wynikDecFun)  
    return testujDecisionFunction(xTest, yTest, najDecFun, clf)

def stworzGrupowanie(kmeans, data, dataClass, k, p):
    size = data.shape[0]
    arr = np.arange(size)
    err = 0
    n = int(size / (k+1))
    trainCount = int(2 * n * p)
    wynik = []
    for i in range(k):
        trainMask = np.arange(i*n, i*n + trainCount)
        testMask = np.arange(i*n + trainCount, (i+2)*n)
        y_train = dataClass[trainMask]
        y_test = dataClass[testMask]  
        
        xKmeansTrain = data[trainMask]
        xKmeansTest = data[testMask]
        kmeans.fit(xKmeansTrain)
        trainLabels = kmeans.labels_
        testLabels = kmeans.predict(xKmeansTest)
        wynik.append([trainLabels, testLabels])
    return wynik

def test(nazwa,listaTablic, kmeans, clf, tablicaDoKmeans, rodzajDanychDoKmeans, liczbaKolumnKmeans = 60, ilosc = 4, p = 0.5, liczbaKolumn = 60):
    kmeansData = stworzGrupowanie(kmeans, (((tablicaDoKmeans[rodzajDanychDoKmeans]).T[:liczbaKolumnKmeans]).T), zmiana[oIndeksy], ilosc, p)
    wynik1 = 0
    wynik2 = 0
    wynik3 = 0
    wynik4 = 0
    wynik5 = 0
    for i in range(5):  
        wynik1 += crossValidationGroupingPoKolei(clf, (((listaTablic[0]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, kmeansData)
        wynik2 += crossValidationGroupingPoKolei(clf, (((listaTablic[1]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, kmeansData)
        wynik3 += crossValidationGroupingPoKolei(clf, (((listaTablic[2]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, kmeansData)
        wynik4 += crossValidationGroupingPoKolei(clf, (((listaTablic[3]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, kmeansData)
        wynik5 += crossValidationGroupingPoKolei(clf, (((listaTablic[4]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, kmeansData)
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1/5, wynik2/5, wynik3/5, wynik4/5, wynik5/5), " --- rodzaj danych kmeans: ", rodzaj(rodzajDanychDoKmeans))
    
def testBezKmeans(nazwa,listaTablic, kmeans, clf, tablicaDoKmeans, rodzajDanychDoKmeans, liczbaKolumnKmeans = 60, ilosc = 4, p = 0.5, liczbaKolumn = 60):
    wynik1 = crossValidationPoKolei(clf, (((listaTablic[0]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p)
    wynik2 = crossValidationPoKolei(clf, (((listaTablic[1]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p)
    wynik3 = crossValidationPoKolei(clf, (((listaTablic[2]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p)
    wynik4 = crossValidationPoKolei(clf, (((listaTablic[3]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p)
    wynik5 = crossValidationPoKolei(clf, (((listaTablic[4]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p)
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5), " --- bez KMeans")
    
def duzyTest(nazwaKmeans, nazwa, listaTablic, kmeans, clf, tablicaDoKmeans, liczbaKolumn = 60, ilosc = 4, p = 0.5):
    print('Czyste    S       N      S->N    N->S      ------ ', nazwaKmeans)
    for rodz in range(5):
        test("imbalance", listaTablic, kmeans, clf, tablicaDoKmeans, rodz, liczbaKolumn, ilosc, p, liczbaKolumn)
    print()

In [12]:
def rodzaj(x):
    return {
        0 : 'Czyste',
        1 : 'S',
        2 : 'N',
        3 : 'S->N',
        4 : 'N->S'
    }[x]

In [15]:
tablicaCalekRosnaca.shape

(5474, 68)

In [15]:
def bardzoDuzyTest(kmeans, svm):
    print('Czyste    S       N      S->N    N->S ------------ kmeans.k :', kmeans.n_clusters)
    testBezKmeans("imbalance", oListaImbalance, kmeans, svm, oListaImbalance, 0, 120, 4, 0.7, 120)
    
    duzyTest("imbalance", "imbalance", oListaImbalance, kmeans, svm, oListaImbalance, liczbaKolumn = 80, ilosc = 4, p = 0.7)
    duzyTest("tablicaTickowPo1[60]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaTickowPo1, liczbaKolumn = 60, ilosc = 4, p = 0.7)
    duzyTest("tablicaTickowPo1[90]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaTickowPo1, liczbaKolumn = 90, ilosc = 4, p = 0.7)
    duzyTest("tablicaTickowPo1[120]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaTickowPo1, liczbaKolumn = 120, ilosc = 4, p = 0.7)
    duzyTest("tablicaTickowRosnaca", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaTickowRosnaca, liczbaKolumn = 68, ilosc = 4, p = 0.7)
    duzyTest("tablicaCalekPo1[60]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaCalekPo1, liczbaKolumn = 60, ilosc = 4, p = 0.7)
    duzyTest("tablicaCalekPo1[90]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaCalekPo1, liczbaKolumn = 90, ilosc = 4, p = 0.7)
    duzyTest("tablicaCalekPo1[120]", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaCalekPo1, liczbaKolumn = 120, ilosc = 4, p = 0.7)
    duzyTest("tablicaCalekRosnaca", "imbalance", oListaImbalance, kmeans, svm, oListaTablicaCalekRosnaca, liczbaKolumn = 68, ilosc = 4, p = 0.7)

In [16]:
kmeans = KMeans(init='k-means++', n_clusters=4, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 4
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5316  0.5316  0.5072  0.5068  0.5072  --- rodzaj danych kmeans:  Czyste
0.5316  0.5316  0.5072  0.5068  0.5072  --- rodzaj danych kmeans:  S
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N
0.4693  0.4693  0.4644  0.4328  0.4644  --- rodzaj danych kmeans:  S->N
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5221  0.5221  0.5370  0.5370  0.5370  --- rodzaj danych kmeans:  Czyste
0.5275  0.5275  0.5334  0.5275  0.5334  --- rodzaj danych kmeans:  S
0.5298  0.5298  0.5302  0.5311  0.5302  --- rodzaj danych kmeans:  N
0.5379  0.5379  0.5293  0.5289  0.5293  --- rodzaj danych kmeans:  S->N
0.5171  0.5171  0.5293  0.5275  0.5293  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [17]:
kmeans = KMeans(init='k-means++', n_clusters=4, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 4
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5302  0.5302  0.5077  0.5068  0.5077  --- rodzaj danych kmeans:  Czyste
0.5302  0.5302  0.5077  0.5068  0.5077  --- rodzaj danych kmeans:  S
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N
0.4693  0.4693  0.4644  0.4328  0.4644  --- rodzaj danych kmeans:  S->N
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5352  0.5352  0.5338  0.5325  0.5338  --- rodzaj danych kmeans:  Czyste
0.5289  0.5289  0.5271  0.5271  0.5271  --- rodzaj danych kmeans:  S
0.5293  0.5293  0.5307  0.5307  0.5307  --- rodzaj danych kmeans:  N
0.5334  0.5334  0.5293  0.5289  0.5293  --- rodzaj danych kmeans:  S->N
0.5293  0.5293  0.5262  0.5262  0.5262  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [18]:
kmeans = KMeans(init='k-means++', n_clusters=3, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 3
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5253  0.5253  0.5041  0.5027  0.5041  --- rodzaj danych kmeans:  Czyste
0.5253  0.5253  0.5041  0.5027  0.5041  --- rodzaj danych kmeans:  S
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N
0.5384  0.5384  0.5338  0.5023  0.5338  --- rodzaj danych kmeans:  S->N
0.4797  0.4797  0.4567  0.4567  0.4567  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5366  0.5366  0.5338  0.5325  0.5338  --- rodzaj danych kmeans:  Czyste
0.5329  0.5329  0.5280  0.5280  0.5280  --- rodzaj danych kmeans:  S
0.5388  0.5388  0.5293  0.5293  0.5293  --- rodzaj danych kmeans:  N
0.5280  0.5280  0.5289  0.5284  0.5289  --- rodzaj danych kmeans:  S->N
0.5352  0.5352  0.5307  0.5293  0.5307  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [19]:
kmeans = KMeans(init='k-means++', n_clusters=5, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 5
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5140  0.5140  0.5041  0.5045  0.5041  --- rodzaj danych kmeans:  Czyste
0.5140  0.5140  0.5041  0.5045  0.5041  --- rodzaj danych kmeans:  S
0.4188  0.4188  0.4016  0.4016  0.4016  --- rodzaj danych kmeans:  N
0.4693  0.4693  0.4644  0.4328  0.4644  --- rodzaj danych kmeans:  S->N
0.4188  0.4188  0.4016  0.4016  0.4016  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5275  0.5275  0.5375  0.5375  0.5375  --- rodzaj danych kmeans:  Czyste
0.5239  0.5239  0.5284  0.5280  0.5284  --- rodzaj danych kmeans:  S
0.5275  0.5275  0.5352  0.5361  0.5352  --- rodzaj danych kmeans:  N
0.5402  0.5402  0.5343  0.5325  0.5343  --- rodzaj danych kmeans:  S->N
0.5167  0.5167  0.5275  0.5275  0.5275  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [20]:
kmeans = KMeans(init='k-means++', n_clusters=7, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 7
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5194  0.5194  0.4991  0.4982  0.4991  --- rodzaj danych kmeans:  Czyste
0.5086  0.5086  0.4995  0.4973  0.4995  --- rodzaj danych kmeans:  S
0.1435  0.1435  0.1449  0.1449  0.1449  --- rodzaj danych kmeans:  N
0.2473  0.2473  0.2414  0.2171  0.2414  --- rodzaj danych kmeans:  S->N
0.3294  0.3294  0.2978  0.2978  0.2978  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5162  0.5162  0.5370  0.5366  0.5370  --- rodzaj danych kmeans:  Czyste
0.5248  0.5248  0.5289  0.5289  0.5289  --- rodzaj danych kmeans:  S
0.5203  0.5203  0.5361  0.5356  0.5361  --- rodzaj danych kmeans:  N
0.5352  0.5352  0.5262  0.5253  0.5262  --- rodzaj danych kmeans:  S->N
0.5153  0.5153  0.5253  0.5235  0.5253  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [21]:
kmeans = KMeans(init='k-means++', n_clusters=9, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 9
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5045  0.5045  0.4964  0.4977  0.4964  --- rodzaj danych kmeans:  Czyste
0.4964  0.4964  0.4914  0.4941  0.4914  --- rodzaj danych kmeans:  S
0.1435  0.1435  0.1449  0.1449  0.1449  --- rodzaj danych kmeans:  N
0.2473  0.2473  0.2414  0.2171  0.2414  --- rodzaj danych kmeans:  S->N
0.3294  0.3294  0.2978  0.2978  0.2978  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5266  0.5266  0.5338  0.5338  0.5338  --- rodzaj danych kmeans:  Czyste
0.5257  0.5257  0.5284  0.5280  0.5284  --- rodzaj danych kmeans:  S
0.5266  0.5266  0.5320  0.5320  0.5320  --- rodzaj danych kmeans:  N
0.5289  0.5289  0.5257  0.5262  0.5257  --- rodzaj danych kmeans:  S->N
0.5262  0.5262  0.5199  0.5190  0.5199  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N

In [22]:
kmeans = KMeans(init='k-means++', n_clusters=15, n_init=5)
svm = SVC(class_weight='balanced', gamma = 1e-3, kernel = 'rbf', C = 1, tol = 1e-3, cache_size=4000)

t = time.time()
bardzoDuzyTest(kmeans, svm)
print(time.time() - t)

Czyste    S       N      S->N    N->S ------------ kmeans.k : 15
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
Czyste    S       N      S->N    N->S      ------  imbalance
0.5108  0.5108  0.4932  0.4959  0.4932  --- rodzaj danych kmeans:  Czyste
0.5162  0.5162  0.4964  0.5014  0.4964  --- rodzaj danych kmeans:  S
0.1435  0.1435  0.1449  0.1449  0.1449  --- rodzaj danych kmeans:  N
0.2473  0.2473  0.2414  0.2171  0.2414  --- rodzaj danych kmeans:  S->N
0.3294  0.3294  0.2978  0.2978  0.2978  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    N->S      ------  tablicaTickowPo1[60]
0.5045  0.5045  0.5203  0.5194  0.5203  --- rodzaj danych kmeans:  Czyste
0.5271  0.5271  0.5149  0.5126  0.5149  --- rodzaj danych kmeans:  S
0.5203  0.5203  0.5104  0.5117  0.5104  --- rodzaj danych kmeans:  N
0.5194  0.5194  0.5190  0.5158  0.5190  --- rodzaj danych kmeans:  S->N
0.5158  0.5158  0.5167  0.5126  0.5158  --- rodzaj danych kmeans:  N->S

Czyste    S       N      S->N    

In [232]:
duzyTest("imbalance", oListaImbalance, kmeans, svm, oListaImbalance, p = 0.7)

Czyste    S       N      S->N    N->S
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
0.5212  0.5212  0.5158  0.5185  0.5153  --- rodzaj danych kmeans:  Czyste
0.5054  0.5054  0.5135  0.5140  0.5131  --- rodzaj danych kmeans:  S
0.3524  0.3524  0.3633  0.3443  0.3425  --- rodzaj danych kmeans:  N
0.4653  0.4653  0.4440  0.4156  0.4440  --- rodzaj danych kmeans:  S->N
0.3010  0.3010  0.3114  0.2893  0.2699  --- rodzaj danych kmeans:  N->S


In [234]:
duzyTest("imbalance", oListaImbalance, kmeans, svm, oListaTablicaCalekRosnaca, liczbaKolumn = 100, p = 0.7)

Czyste    S       N      S->N    N->S
0.5393  0.5393  0.5338  0.5325  0.5338  --- bez KMeans
0.5311  0.5311  0.5307  0.5271  0.5307  --- rodzaj danych kmeans:  Czyste
0.5162  0.5162  0.5253  0.5262  0.5253  --- rodzaj danych kmeans:  S
0.5311  0.5311  0.5334  0.5325  0.5334  --- rodzaj danych kmeans:  N
0.5271  0.5271  0.5388  0.5375  0.5388  --- rodzaj danych kmeans:  S->N
0.5329  0.5329  0.5298  0.5298  0.5298  --- rodzaj danych kmeans:  N->S


In [248]:
svm.classes_

array([-1,  1])

In [266]:
np.unique(np.array([1,2,3,4,3,21]))

array([ 1,  2,  3,  4, 21])

In [31]:
def test(nazwa,listaTablic, kmeans, clf, listaTablicKmeans, ilosc = 4, p = 0.5, liczbaKolumn = 60):
    print(nazwa, ', dla procent =', p)
    wynik1 = crossValidationGroupingPoKolei(clf, (((listaTablic[0]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, listaTablicKmeans[0])
    wynik2 = crossValidationGroupingPoKolei(clf, (((listaTablic[1]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, listaTablicKmeans[0])
    wynik3 = crossValidationGroupingPoKolei(clf, (((listaTablic[2]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, listaTablicKmeans[0])
    wynik4 = crossValidationGroupingPoKolei(clf, (((listaTablic[3]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, listaTablicKmeans[0])
    wynik5 = crossValidationGroupingPoKolei(clf, (((listaTablic[4]).T[:liczbaKolumn]).T), zmiana[oIndeksy], ilosc, p, kmeans, listaTablicKmeans[0])
    print('Czyste    S       N      S->N    N->S')
    print("{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5))

(5474, 1)